<a href="https://colab.research.google.com/github/G2TECH/-LGMVIP--DataScience/blob/Beginner-Task/task_2_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df = pd.read_csv(url)

In [ ]:
# Extract the 'Close' column for prediction
data = df.filter(['Close']).values
dataset = data.astype('float32')


In [ ]:
# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
# Split the dataset into training and testing sets
training_data_len = int(len(dataset) * 0.8)
train_data = scaled_data[:training_data_len, :]
test_data = scaled_data[training_data_len:, :]

In [ ]:
# Define the function to create the training dataset
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps, 0])
        y.append(data[i+time_steps, 0])
    return np.array(X), np.array(y)


In [ ]:
# Set the time steps
time_steps = 60


In [ ]:
# Create the training dataset
X_train, y_train = create_dataset(train_data, time_steps)

# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
98/98 [==============================] - 9s 47ms/step - loss: 0.0045
Epoch 2/10
98/98 [==============================] - 4s 42ms/step - loss: 8.7384e-04
Epoch 3/10
98/98 [==============================] - 6s 64ms/step - loss: 8.7880e-04
Epoch 4/10
98/98 [==============================] - 4s 40ms/step - loss: 7.4876e-04
Epoch 5/10
98/98 [==============================] - 4s 41ms/step - loss: 6.9355e-04
Epoch 6/10
98/98 [==============================] - 6s 60ms/step - loss: 6.1629e-04
Epoch 7/10
98/98 [==============================] - 5s 51ms/step - loss: 5.6419e-04
Epoch 8/10
98/98 [==============================] - 4s 41ms/step - loss: 6.9138e-04
Epoch 9/10
98/98 [==============================] - 6s 58ms/step - loss: 4.8954e-04
Epoch 10/10
98/98 [==============================] - 4s 40ms/step - loss: 5.3989e-04


In [ ]:
# Create the testing dataset
X_test, y_test = create_dataset(test_data, time_steps)


In [ ]:
# Reshape the input data for LSTM
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions on the testing dataset
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

11/11 [==============================] - 2s 25ms/step


In [ ]:
# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 101.34133911132812
